# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [90]:
# Import libraries
import re
import numpy as np
import time
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.preprocessing import OneHotEncoder
s_words=set(stopwords.words('english'))

In [6]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\stone\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\stone\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\stone\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [81]:
# Load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('SELECT * FROM cleaned_msg',engine).drop('original',axis=1)
X = df.iloc[:,0:4]
Y = df.iloc[:,4:]
Y.head()

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [62]:
def tokenize(text):
    text = re.sub('[^\w]', " ",text)

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [86]:
# vect = TfidfVectorizer(tokenizer=tokenize)
# onehot=OneHotEncoder()
# clf=MultiOutputClassifier(XGBClassifier(random_state=42,n_jobs=-1))

# preprocessor=ColumnTransformer(transformers=[('vect',vect,['message']),
#                                      ('encoder',onehot,['genre','related']),
#                                     ],remainder='drop')

pipe=Pipeline(steps=[('vect',TfidfVectorizer(tokenizer=tokenize)),
                    ('clf',MultiOutputClassifier(XGBClassifier(random_state=42,n_jobs=-1,objective='multi:softmax',
                                                              num_class=35,eval_metric='merror',use_label_encoder=False)))
                    ])

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x000001648643D940>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                                                      booster=None,
                                                                                      colsample_bylevel=None,
                                                                                      colsample_bynode=None,
                                                                                      colsample_bytree=None,
                                                                                      eval_metric='merror',
                                                                                      gamma=None,
                                                         

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [88]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

start_time=time.time()
pipe.fit(X_train['message'],y_train)
print('training_time:',time.time()-start_time)

training_time: 710.5121474266052


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [98]:
categories=Y.columns
pred=pipe.predict(X_test['message'])
for i,col in enumerate(categories):
    print(f'classification report for {col}')
    print(classification_report(y_test.iloc[:,i],pred[:,i]))
    print('******************')

classification report for request
              precision    recall  f1-score   support

           0       0.91      0.96      0.94      4319
           1       0.75      0.58      0.65       925

    accuracy                           0.89      5244
   macro avg       0.83      0.77      0.80      5244
weighted avg       0.89      0.89      0.89      5244

******************
classification report for offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5222
           1       0.00      0.00      0.00        22

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.99      1.00      0.99      5244

******************
classification report for aid_related
              precision    recall  f1-score   support

           0       0.78      0.85      0.81      3054
           1       0.76      0.66      0.71      2190

    accuracy                           0

C:\Users\stone\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stone\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stone\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.98      0.99      0.98      5014
           1       0.69      0.50      0.58       230

    accuracy                           0.97      5244
   macro avg       0.83      0.74      0.78      5244
weighted avg       0.96      0.97      0.97      5244

******************
classification report for other_aid
              precision    recall  f1-score   support

           0       0.89      0.97      0.93      4554
           1       0.48      0.17      0.25       690

    accuracy                           0.87      5244
   macro avg       0.68      0.57      0.59      5244
weighted avg       0.83      0.87      0.84      5244

******************
classification report for infrastructure_related
              precision    recall  f1-score   support

           0       0.94      0.99      0.96      4904
           1       0.36      0.09      0.15       340

    accuracy                           0.93      5244
   ma

C:\Users\stone\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stone\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\stone\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters ={ 
    #'clf__estimator__max_depth':[3,5,7],
         #'clf__estimator__learning_rate':[0.01,0.015,0.02,0.05,0.1],
         'clf__estimator__gamma':[0.05,0.1,0.3],
         #'clf__estimator__min_child_weight':[1,3,5],
         #'clf__estimator__subsample':[0.6,0.8],
         #'clf__estimator__colsample_bytree':[0.6,0.8],
         #'clf__estimator__reg_alpha':[0.01,0.1,0.5,1,3],
         'clf__estimator__reg_lambda':[0.1,0.5,1]}


cv = GridSearchCV(pipe,param_grid=params)

start_time=time.time()
cv.fit(X_train['message'],y_train)
print('training_time:',time.time()-start_time)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [97]:
a1=np.array([[2,3,5],[4,7,1]])
a1[:,1]

array([3, 7])